In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
from feedback.generate_report import *
import feedback.secret as secret
from feedback.database import open_connection
import pandas as pd

In [15]:
start_date = "2024-05-20"
end_date = "2024-05-26"

## Donors

In [5]:
db_name = "fr_new" 
username = secret.database_username 
password = secret.database_password 
ip_address = "localhost"
port = secret.database_port

In [6]:
connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [7]:
# feedbacks = get_feedback_by_date(connection,start_date,end_date)
# feedbacks.to_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

/usr0/home/naveenr/projects/food_rescue_feedback/feedback/database.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
/usr0/home/naveenr/projects/food_rescue_feedback/feedback/database.py:59: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [8]:
# annotated_feedback = generate_prompts_and_analyze_feedback(feedbacks)
# annotated_feedback.to_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)

On feedback 1 out of 689
On feedback 2 out of 689
On feedback 3 out of 689
On feedback 4 out of 689
On feedback 5 out of 689
On feedback 6 out of 689
On feedback 7 out of 689


KeyboardInterrupt: 

In [16]:
feedbacks = pd.read_csv('../../results/reports/feedbacks_{}_{}.csv'.format(start_date,end_date))
annotated_feedback = pd.read_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date))

In [17]:
# Calculate statistics and generate the report
donor_statistics = calculate_donor_statistics(feedbacks, annotated_feedback)
recipient_statistics = calculate_recipient_statistics(feedbacks, annotated_feedback)
report_info = info_update(annotated_feedback)

# Save the report to a CSV file
donor_statistics.to_csv('../../results/reports/report_donor_{}_{}.csv'.format(start_date,end_date), index=False)
recipient_statistics.to_csv('../../results/reports/report_recipient_{}_{}.csv'.format(start_date,end_date), index=False)
report_info.to_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date), index=False)